For all workers that work out of the home, the telecommute models predicts the level of telecommuting. The model alternatives are the frequency of telecommuting in days per week (0 days, 1 day, 2 to 3 days, 4+ days).

Telecommuting is defined as workers who work from home instead of going to work. It only applies to workers with a regular workplace outside of home.

In [1]:
import os
import toml
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color"



In [2]:
# get shared data
%store -r validation_data

#config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

per_data = validation_data.persons_data_uncloned.copy()
hh_data = validation_data.hh_data_uncloned.copy()
land_use = validation_data.land_use[['zone_id', 'TAZ', 'county_id']].copy()

# delete shared data
del validation_data



In [3]:
income_4group = hh_data.loc[hh_data['source'] == 'model results',['income']].quantile([.25, .50, .75])
# add income group
hh_data['hhincome_group'] = pd.cut(hh_data['income'],bins=[-9999999.0] + income_4group['income'].tolist() + [9999999.0], labels=['low', 'medium', 'medium-high', 'high'])

telecommute_cat = {"No_Telecommute": "0 day",
                   "1_day_week": "1 day",
                   "2_3_days_week": "2-3 days",
                   "4_days_week": "4 days"}
per_data['telecommute_frequency_des'] = per_data['telecommute_frequency'].map(telecommute_cat)


In [4]:
per_data = per_data.merge(land_use,how="left",left_on='home_zone_id',right_on='zone_id')
per_data = per_data.merge(hh_data[['source','household_id','hhincome_group']],how="left",on=['source','household_id'])
per_data = per_data.loc[(per_data['pemploy']<3) & (~per_data['work_from_home'])]


In [5]:
df_plot = per_data.groupby(['source','telecommute_frequency_des'])[['person_weight']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['person_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="telecommute_frequency_des", y="percentage", color="source",barmode="group", 
             title="telecommute frequency<br>for workers with usual work location outside of home",)
fig.update_layout(height=400, width=700, font=dict(size=11), yaxis=dict(tickformat=".2%"))
fig.show()

## telecommute frequency by segment

In [6]:
# auto ownership in Income groups
def plot_segments(df:pd.DataFrame, var:str, title_cat:str,sub_name:str):
    # print(f"n=\n"
    #       f"{df.loc[df['source']=='model results',var].value_counts()[df[var].sort_values().unique()]}")
    df_plot = df.groupby(['source',var,'telecommute_frequency_des'])['person_weight'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',var], group_keys=False)['person_weight'].\
        apply(lambda x: 100 * x / float(x.sum()))

    fig = px.bar(df_plot, x="telecommute_frequency_des", y="percentage", color="source",
                 facet_col=var, barmode="group",
                 title="telecommute frequency by "+ title_cat)
    fig.for_each_annotation(lambda a: a.update(text = sub_name + "=<br>" + a.text.split("=")[-1]))
    fig.update_xaxes(title_text="")
    fig.update_layout(height=400, width=800, font=dict(size=11))
    fig.show()

In [7]:
plot_segments(per_data,'hhincome_group','Income', 'Income')

C:\Users\Modeller\AppData\Local\Temp\ipykernel_12036\3791975383.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Modeller\AppData\Local\Temp\ipykernel_12036\3791975383.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [8]:
plot_segments(per_data,'county_id','County', 'County')